In [1]:
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import plotly.express as px
import base64
from requests import post, get
import json
import csv
from sklearn import preprocessing

In [2]:
client_id = 'f177c75faa7d4fba999b81b922c1a042'
client_secret = '46d8af627ab54f5a9462a50a6505ee6e'
playlistId = '0IN7IWKmIfwlEysGyWUuRg'

dataset = []
dataset2 = []
dataset3 = []

In [3]:
def getToken():
    auth_string = client_id + ':' + client_secret

    auth_b64 = base64.b64encode(auth_string.encode('utf-8'))

    url = 'https://accounts.spotify.com/api/token'

    headers = {
        'Authorization': 'Basic ' + auth_b64.decode('utf-8'),
        'Content-Type': 'application/x-www-form-urlencoded'
    }

    data = {'grant_type': 'client_credentials'}

    result = post(url, headers=headers, data=data)

    json_result = json.loads(result.content)
    token = json_result['access_token']

    return token

In [4]:
def getAuthHeader(token):
    return {'Authorization': 'Bearer ' + token}

In [5]:
def getAudioFeatures(token, trackId):

    url = f'https://api.spotify.com/v1/audio-features/{trackId}'

    headers = getAuthHeader(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)

    audio_features_temp = [
        json_result['danceability'],
        json_result['energy'],
        json_result['key'],
        json_result['loudness'],
        json_result['mode'],
        json_result['speechiness'],
        json_result['acousticness'],
        json_result['instrumentalness'],
        json_result['liveness'],
        json_result['valence'],
        json_result['tempo'],
    ]
    dataset2.append(audio_features_temp)

In [7]:
def getPlaylistItems(token, playlistId):

    url = f'https://api.spotify.com/v1/playlists/{playlistId}/tracks'
    limit = '&limit=100'
    market = '?market=ID'

    fields = '&fields=items%28track%28id%2Cname%2Cartists%2Cpopularity%2C+duration_ms%2C+album%28release_date%29%29%29'
    url = url+market+fields+limit

    headers = getAuthHeader(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)

    for i in range(len(json_result['items'])):
        playlist_items_temp = []
        playlist_items_temp.append(json_result['items'][i]['track']['id'])
        playlist_items_temp.append(
            json_result['items'][i]['track']['name'].encode('utf-8'))
        playlist_items_temp.append(
            json_result['items'][i]['track']['artists'][0]['name'].encode('utf-8'))
        playlist_items_temp.append(
            json_result['items'][i]['track']['popularity'])
        playlist_items_temp.append(
            json_result['items'][i]['track']['duration_ms'])
        playlist_items_temp.append(
            int(json_result['items'][i]['track']['album']['release_date'][0:4]))
        dataset.append(playlist_items_temp)

    for i in range(len(json_result['items'])):
        playlist_items_temp = []
        playlist_items_temp.append(json_result['items'][i]['track']['id'])
        playlist_items_temp.append(
            json_result['items'][i]['track']['name'].encode('utf-8'))
        playlist_items_temp.append(
            json_result['items'][i]['track']['artists'][0]['name'].encode('utf-8'))
        playlist_items_temp.append(
            json_result['items'][i]['track']['popularity'])
        playlist_items_temp.append(
            json_result['items'][i]['track']['duration_ms'])
        playlist_items_temp.append(
            int(json_result['items'][i]['track']['album']['release_date'][0:4]))
        dataset.append(playlist_items_temp)

    for i in range(len(dataset)):
        getAudioFeatures(token, dataset[i][0])

    for i in range(len(dataset)):
        dataset3.append(dataset[i]+dataset2[i])

    print(dataset3)
    
    with open('dataset.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["id", "name", "artist", "popularity", "duration_ms", "year", "danceability", "energy", "key", "loudness", "mode",
                         "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"])
        writer.writerows(dataset3)

In [8]:
token = getToken() 
print('access token : '+token) 
getPlaylistItems(token, playlistId)

access token : BQBTrBovl04DnrKv57zvTalg_3gRvFJZionoySakxK1hcaPHUa_vZSE5mVCRRFozOFPCsx6MMAf1B40xrG1AetDy225v0afFeZg0BtF13H6miwWsjcA
[['14juitXdt9JqvBQAQP3i5L', b'Part Time God', b'The Bad Dreamers', 38, 217683, 2018, 0.682, 0.669, 10, -8.068, 0, 0.0803, 0.0471, 0.000484, 0.277, 0.548, 98.005], ['6HXVNP9mGsbTKG6ovKLzcH', b'Fringe Society', b'Sami Matar', 39, 222851, 2017, 0.504, 0.294, 8, -9.847, 1, 0.0296, 0.000748, 0.768, 0.172, 0.0382, 80.029], ['5sPkgogsCQFYKnYZMwkODz', b'The Arcane Arts', b'Arcis', 34, 172109, 2017, 0.638, 0.899, 8, -7.013, 1, 0.0774, 0.0001, 0.829, 0.118, 0.343, 114.993], ['4YyG15mMNRqjYCH3S7aBw0', b'Knightriders', b'Fury Weekend', 35, 281379, 2017, 0.575, 0.934, 2, -3.641, 0, 0.0441, 2.64e-05, 0.856, 0.147, 0.475, 116.018], ['0neQGBPQlP3WPPv1zKEyXU', b'WAITING', b'Kayzo', 26, 251200, 2018, 0.508, 0.929, 8, -3.018, 1, 0.142, 0.00099, 0.313, 0.0876, 0.227, 149.844], ['0UeYCHOETPfai02uskjJ3x', b'Shooting Stars', b'Bag Raiders', 67, 235813, 2011, 0.636, 0.72, 11, -6.1

In [9]:
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import plotly.express as px 

data = pd.read_csv('dataset.csv')
data.head()

,id,name,artist,popularity,duration_ms,year,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,14juitXdt9JqvBQAQP3i5L,b'Part Time God',b'The Bad Dreamers',38,217683,2018,0.682,0.669,10,-8.068,0,0.0803,0.047100,0.000484,0.2770,0.5480,98.005
1,6HXVNP9mGsbTKG6ovKLzcH,b'Fringe Society',b'Sami Matar',39,222851,2017,0.504,0.294,8,-9.847,1,0.0296,0.000748,0.768000,0.1720,0.0382,80.029
2,5sPkgogsCQFYKnYZMwkODz,b'The Arcane Arts',b'Arcis',34,172109,2017,0.638,0.899,8,-7.013,1,0.0774,0.000100,0.829000,0.1180,0.3430,114.993
3,4YyG15mMNRqjYCH3S7aBw0,b'Knightriders',b'Fury Weekend',35,281379,2017,0.575,0.934,2,-3.641,0,0.0441,0.000026,0.856000,0.1470,0.4750,116.018
4,0neQGBPQlP3WPPv1zKEyXU,b'WAITING',b'Kayzo',26,251200,2018,0.508,0.929,8,-3.018,1,0.1420,0.000990,0.313000,0.0876,0.2270,149.844


In [10]:
data['artist'] = data['artist'].map(lambda x: str(x)[2:-1])
data['name'] = data['name'].map(lambda x: str(x)[2:-1])

data.head()

,id,name,artist,popularity,duration_ms,year,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,14juitXdt9JqvBQAQP3i5L,Part Time God,The Bad Dreamers,38,217683,2018,0.682,0.669,10,-8.068,0,0.0803,0.047100,0.000484,0.2770,0.5480,98.005
1,6HXVNP9mGsbTKG6ovKLzcH,Fringe Society,Sami Matar,39,222851,2017,0.504,0.294,8,-9.847,1,0.0296,0.000748,0.768000,0.1720,0.0382,80.029
2,5sPkgogsCQFYKnYZMwkODz,The Arcane Arts,Arcis,34,172109,2017,0.638,0.899,8,-7.013,1,0.0774,0.000100,0.829000,0.1180,0.3430,114.993
3,4YyG15mMNRqjYCH3S7aBw0,Knightriders,Fury Weekend,35,281379,2017,0.575,0.934,2,-3.641,0,0.0441,0.000026,0.856000,0.1470,0.4750,116.018
4,0neQGBPQlP3WPPv1zKEyXU,WAITING,Kayzo,26,251200,2018,0.508,0.929,8,-3.018,1,0.1420,0.000990,0.313000,0.0876,0.2270,149.844


In [11]:
data = data[data['name'] != '']

data = data.reset_index(drop=True)
data.head()

,id,name,artist,popularity,duration_ms,year,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,14juitXdt9JqvBQAQP3i5L,Part Time God,The Bad Dreamers,38,217683,2018,0.682,0.669,10,-8.068,0,0.0803,0.047100,0.000484,0.2770,0.5480,98.005
1,6HXVNP9mGsbTKG6ovKLzcH,Fringe Society,Sami Matar,39,222851,2017,0.504,0.294,8,-9.847,1,0.0296,0.000748,0.768000,0.1720,0.0382,80.029
2,5sPkgogsCQFYKnYZMwkODz,The Arcane Arts,Arcis,34,172109,2017,0.638,0.899,8,-7.013,1,0.0774,0.000100,0.829000,0.1180,0.3430,114.993
3,4YyG15mMNRqjYCH3S7aBw0,Knightriders,Fury Weekend,35,281379,2017,0.575,0.934,2,-3.641,0,0.0441,0.000026,0.856000,0.1470,0.4750,116.018
4,0neQGBPQlP3WPPv1zKEyXU,WAITING,Kayzo,26,251200,2018,0.508,0.929,8,-3.018,1,0.1420,0.000990,0.313000,0.0876,0.2270,149.844


In [12]:
data2 = data.copy()
data2 = data2.drop(['artist', 'name', 'year', 'popularity', 'key','duration_ms', 'mode', 'id'], axis=1)

data2.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.682,0.669,-8.068,0.0803,0.047100,0.000484,0.2770,0.5480,98.005
1,0.504,0.294,-9.847,0.0296,0.000748,0.768000,0.1720,0.0382,80.029
2,0.638,0.899,-7.013,0.0774,0.000100,0.829000,0.1180,0.3430,114.993
3,0.575,0.934,-3.641,0.0441,0.000026,0.856000,0.1470,0.4750,116.018
4,0.508,0.929,-3.018,0.1420,0.000990,0.313000,0.0876,0.2270,149.844


In [13]:
from sklearn import preprocessing

x = data2.values 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
data2 = pd.DataFrame(x_scaled)

data2.columns = ['acousticness','danceability','energy','instrumentalness','loudness', 'liveness', 'speechiness', 'tempo','valence']

data2.describe()

,acousticness,danceability,energy,instrumentalness,loudness,liveness,speechiness,tempo,valence
count,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000
mean,0.586685,0.728053,0.751623,0.108455,0.092037,0.411947,0.236606,0.402855,0.453614
std,0.250737,0.241278,0.158212,0.163875,0.182343,0.411207,0.218415,0.266717,0.249533
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.430702,0.608108,0.670483,0.024191,0.001032,0.000341,0.079744,0.212996,0.297312
50%,0.639474,0.798009,0.766444,0.046533,0.005295,0.325077,0.162756,0.332977,0.399036
75%,0.801754,0.914651,0.854643,0.119414,0.112199,0.871775,0.387362,0.596698,0.597807
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
pca = PCA(n_components=2)
pca.fit(data2)
pca_data = pca.transform(data2)

pca_df = pd.DataFrame(data=pca_data, columns=['x', 'y'])
pca_df.head()

fig = px.scatter(pca_df, x='x', y='y', title='PCA')
fig.show()

In [15]:
data2 = list(zip(pca_df['x'], pca_df['y']))

kmeans = KMeans(n_init=10, max_iter=1000).fit(data2)

fig = px.scatter(pca_df, x='x', y='y', color=kmeans.labels_, color_continuous_scale='rainbow', hover_data=[data.artist, data.name])
fig.show()

In [16]:
def dataProcessing():
    data = pd.read_csv('dataset.csv')
    data
    st.write("## Preprocessing Result")

    data = data[['artist', 'name', 'year', 'popularity', 'key', 'mode', 'duration_ms', 'acousticness',
                'danceability', 'energy', 'instrumentalness', 'loudness', 'liveness', 'speechiness', 'tempo', 'valence']]
    data = data.drop(['mode'], axis=1)
    data['artist'] = data['artist'].map(lambda x: str(x)[2:-1])
    data['name'] = data['name'].map(lambda x: str(x)[2:-1])
    st.write("### Data to be deleted:")
    data[data['name'] == '']
    data = data[data['name'] != '']

    st.write("## Normalization Result")
    data2 = data.copy()
    data2 = data2.drop(
        ['artist', 'name', 'year', 'popularity', 'key', 'duration_ms'], axis=1)
    x = data2.values
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    data2 = pd.DataFrame(x_scaled)
    data2.columns = ['acousticness', 'danceability', 'energy', 'instrumentalness',
                     'loudness', 'liveness', 'speechiness', 'tempo', 'valence']
    data2

    st.write("## Dimensionality Reduction with PCA")
    pca = PCA(n_components=2)
    pca.fit(data2)
    pca_data = pca.transform(data2)
    pca_df = pd.DataFrame(data=pca_data, columns=['x', 'y'])
    fig = px.scatter(pca_df, x='x', y='y', title='PCA')
    st.plotly_chart(fig)

    st.write("## Clustering with K-Means")
    data2 = list(zip(pca_df['x'], pca_df['y']))
    kmeans = KMeans(n_init=10, max_iter=1000).fit(data2)
    fig = px.scatter(pca_df, x='x', y='y', color=kmeans.labels_,
                     color_continuous_scale='rainbow', hover_data=[data.artist, data.name])
    st.plotly_chart(fig)

    st.write("Enjoy!")

In [17]:
st.write("# Spotify Playlist Clustering")
client_id = st.text_input("Enter Client ID")
client_secret = st.text_input("Enter Client Secret")
playlistId = st.text_input("Enter Playlist ID")


2023-10-05 14:48:18.457 
  command:

    streamlit run C:\Users\ASUS\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [18]:
if st.button('Create Dataset!'):
    token = getToken()
    getPlaylistItems(token, playlistId)